# My first DCGAN 🌱
Deep Convolutional Generative Adversarial Network architecture implementation.
- DCGAN [paper](https://arxiv.org/abs/1511.06434).
- [Papers with Code](https://paperswithcode.com/method/dcgan) explanation.
<details>
<summary>
<font size="3" color="green">
<b>Gan Archtecture Scheme</b>
</font>
</summary>
<div>
<img src = "layers.png" width=800>
</div>

</details>


In [1]:
import torch
from torch import nn
import torch.optim as optim
import torchvision
from torchvision.datasets import MNIST
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from typing import Tuple
import matplotlib.pyplot as plt


class Discriminator(nn.Module):
    def __init__(self, img_channels, hidden_units) -> None:
        """Discriminator model for DCGAN architecture based
         on Unsupervised Representation Learning with Deep Convolutional Generative Adversarial Networks paper.
         Paper: https://arxiv.org/abs/1511.06434

        Args:
            img_channels (int): Number of channels of the image (for example for RGB its 3).
            hidden_units (int): Number of neurons in hidden layer.
        """
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            # First discriminator block (bo batch norm)
            nn.Conv2d(img_channels, hidden_units, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            # Rest of the blocks...
            self._block(in_channels=hidden_units, out_channels=hidden_units * 2),
            self._block(in_channels=hidden_units * 2, out_channels=hidden_units * 4),
            self._block(in_channels=hidden_units * 4, out_channels=hidden_units * 8),
            nn.Conv2d(in_channels=hidden_units * 8, out_channels=1, kernel_size=4, stride=2, padding=0),
            nn.Sigmoid(), # Output range [0, 1]
        )

    def _block(self, in_channels, out_channels, kernel__size=4, stride=2, padding=1) -> torch.Tensor:
        """Creates a single discriminator block consisting of a convolution layer, batch normalization and Leaky ReLU.

        Args:
            in_channels (int): Number of input channels.
                out_channels (int): Number of output channels.
                kernel_size (int, optional): Size of convolution filter. Defaults to 4.
                stride (int, optional): Stride size. Defaults to 2.
                padding (int, optional): Amount of pixels added around the image. Defaults to 1.
        Returns:
            torch.Tensor: Returns a tensor with performed convolutions.
        """
        return nn.Sequential(
            nn.Conv2d(
                in_channels, out_channels, kernel__size, stride, padding, bias=False
            ),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2),
        )

    def forward(self, x: torch.Tensor):
        return self.disc(x)

In [2]:
class Generator(nn.Module):
    def __init__(self, z_dim: int, img_channels: int, hidden_units: int) -> None:
        """_summary_

        Args:
            z_dim (int): Dimension of the noise vector used for image generation.
            img_channels (int): Number of channels in the generated image.
            hidden_units (int): Number of neurons in the hidden layer.
        """
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            # First generator block
            self._generator_block(
                in_channels=z_dim,
                out_channels=hidden_units * 16,
                kernel_size=4,
                stride=1,
                padding=0,
            ),
            # Rest of the generator blocks...
            self._generator_block(
                in_channels=hidden_units * 16, out_channels=hidden_units * 8
            ),
            self._generator_block(
                in_channels=hidden_units * 8, out_channels=hidden_units * 4
            ),
            self._generator_block(
                in_channels=hidden_units * 4, out_channels=hidden_units * 2
            ),
            # Final block
            nn.ConvTranspose2d(
                in_channels=hidden_units * 2,
                out_channels=img_channels,
                kernel_size=4,
                stride=2,
                padding=1,
            ),
            nn.Tanh(),  # Output range: [-1, 1]
        )

    def _generator_block(
        self,
        in_channels: int,
        out_channels: int,
        kernel_size: int = 4,
        stride: int = 2,
        padding: int = 1,
    ) -> torch.Tensor:
        """Creates a single generator block.

        Args:
            in_channels (int): Number of input channels.
            out_channels (int): Number of output channels.
            kernel_size (int, optional): Size of convolution filter. Defaults to 4.
            stride (int, optional): Stride size. Defaults to 2.
            padding (int, optional): Amount of pixels added around the image. Defaults to 1.

        Returns:
            torch.Tensor: Returns a tensor with performed convolutions.
        """
        return nn.Sequential(
            nn.ConvTranspose2d(
                in_channels, out_channels, kernel_size, stride, padding, bias=False
            ),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.gen(x)
            

In [3]:
def initialize_weights(model: nn.Module) -> None:
    """Initializes weights using a normal distribution with mean 0 and std 0.02.

    Args:
        model (nn.Module): The generator or discriminator model.
    """
    for m in model.modules():
        # if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
        if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d)):
            nn.init.normal_(m.weight, 0.0, 0.02)
            if isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.bias, 0)


def test_weights_initialization() -> None:
    """Tests the initialize_weights() function"""
    N, C, H, W = 8, 3, 64, 64
    z_dim = 100

    x = torch.randn(
        (N, C, H, W)
    )  # Simulate a random batch of images of shape N x C x H x W

    ### Test Discriminator
    disc = Discriminator(img_channels=C, hidden_units=8)
    initialize_weights(model=disc)

    # There should be outputet one prediction per image
    assert disc(x).shape == (N, 1, 1, 1), "Discriminators weights are not initialized correctly."

    ### Test Generator
    gen = Generator(z_dim=z_dim, img_channels=C, hidden_units=8)
    initialize_weights(model=gen)
    
    noise = torch.randn((N, z_dim, 1, 1))
    fake_image = gen(noise)
    
    assert fake_image.shape == (
        N,
        C,
        H,
        W,
    ), f"Generators weights are not initialized correctly. Instead of ({N}, {C}, {H}, {W}) they are {fake_image.shape}"

In [4]:
test_weights_initialization()

In [5]:
## Hyperparameters for training
# Setup device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
lr = 2e-4
batch_size = 128
img_size = 64
img_channels = 1
z_dim = 100
num_epochs = 64
hidden_units = 64

transform = transforms.Compose(
    [
        transforms.Resize(img_size),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,)),
    ]
)

# Setup dataset and datalaoder
dataset = MNIST(root='/data', train=True, transform=transform, download=True)
dataloader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)

# Setup Generator
gen = Generator(z_dim=z_dim, img_channels=img_channels, hidden_units=hidden_units).to(device)
initialize_weights(gen)
opt_gen = optim.Adam(gen.parameters(), lr=lr, betas=(0.5, 0.999))

# Setup Discriminator
disc = Discriminator(img_channels=img_channels, hidden_units=hidden_units).to(device)
initialize_weights(disc)
opt_disc = optim.Adam(disc.parameters(), lr=lr, betas=(0.5, 0.999))

# Setup loss
criterion = nn.BCELoss()

# In order to see progress during training for educational purposes
# ( Random noise is generally preferred
#  to promote diversity and robustness in the generated samples).
fixed_noise = torch.randn(batch_size, z_dim, 1, 1).to(device)

# Setup tensorboard (for experiment tracking)
step = 0
writer_fake = SummaryWriter('logs/fake')
writer_real = SummaryWriter('logs/real')

# Set models to train mode
gen.train()
disc.train()

# Train loop
mean_gen_loss, mean_disc_loss = 0, 0


def log_images(real, fake, step, noise: torch.Tensor=fixed_noise) -> None:
    with torch.no_grad():
        fake = gen(noise)

        grid_real = torchvision.utils.make_grid(real[:32], normalize=True)
        writer_real.add_image("Real images", grid_real, global_step=step)

        grid_fake = torchvision.utils.make_grid(fake[:32], normalize=True)
        writer_fake.add_image("Fake (Generated) images", grid_fake, global_step=step)


print(f"DCGAN will be trained on {device}.")
for epoch in range(num_epochs):
    for batch_idx, (real, _) in enumerate(dataloader):
        # Access a batch of real images and move them on the GPU if possible
        real = real.to(device)

        # Generate batch of random noise and move it on the GPU if possible
        noise = torch.randn((batch_size, z_dim, 1, 1)).to(device)
        fake = gen(noise)

        ### Train the Discriminator ###
        disc_real_predicitons = disc(real)#.reshape(-1)# .value(-1)
        disc_fake_predictions = disc(fake)#.reshape(-1)#.value(-1)

        # Calculate the loss
        disc_real_loss = criterion(disc_real_predicitons, torch.ones_like(disc_real_predicitons))
        disc_fake_loss = criterion(disc_fake_predictions, torch.zeros_like(disc_fake_predictions))
        disc_loss = (disc_real_loss + disc_fake_loss) / 2

        # Keep track of the mean discriminator loss
        mean_disc_loss = disc_loss.item() / batch_size


        opt_disc.zero_grad()
        disc_loss.backward(retain_graph=True)
        opt_disc.step()

        ### Train the Generator ###
        disc_fake_pred = disc(fake)
        gen_loss = criterion(disc_fake_pred, torch.ones_like(disc_fake_pred))
        gen.zero_grad()
        gen_loss.backward(retain_graph=True)

        # Keep track of the average generator loss
        mean_gen_loss += gen_loss.item() / batch_size

        if batch_idx % 50 == 0:
            print(
                f"Epoch [{epoch}/{num_epochs}] | Batch {batch_idx}/{len(dataloader)} | Loss Disc: {disc_loss:.4f}, loss Gen: {gen_loss:.4f}"
            )

        # Log results to Tensorboard
        log_images(real=real, fake=fake, noise=fixed_noise, step=step)
        step += 1

DCGAN will be trained on cuda.
Epoch [0/64] | Batch 0/469 | Loss Disc: 0.6950, loss Gen: 0.7717
Epoch [0/64] | Batch 50/469 | Loss Disc: 0.0490, loss Gen: 2.9549
Epoch [0/64] | Batch 100/469 | Loss Disc: 0.0135, loss Gen: 4.1634
Epoch [0/64] | Batch 150/469 | Loss Disc: 0.0061, loss Gen: 4.9695
Epoch [0/64] | Batch 200/469 | Loss Disc: 0.0033, loss Gen: 5.5750
Epoch [0/64] | Batch 250/469 | Loss Disc: 0.0021, loss Gen: 6.0347
Epoch [0/64] | Batch 300/469 | Loss Disc: 0.0015, loss Gen: 6.4137
Epoch [0/64] | Batch 350/469 | Loss Disc: 0.0011, loss Gen: 6.7260
Epoch [0/64] | Batch 400/469 | Loss Disc: 0.0008, loss Gen: 6.9962
Epoch [0/64] | Batch 450/469 | Loss Disc: 0.0006, loss Gen: 7.2354
Epoch [1/64] | Batch 0/469 | Loss Disc: 0.0006, loss Gen: 7.3153
Epoch [1/64] | Batch 50/469 | Loss Disc: 0.0005, loss Gen: 7.5221
Epoch [1/64] | Batch 100/469 | Loss Disc: 0.0004, loss Gen: 7.7031
Epoch [1/64] | Batch 150/469 | Loss Disc: 0.0003, loss Gen: 7.8669
Epoch [1/64] | Batch 200/469 | Loss D

KeyboardInterrupt: 